In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('comp.csv')

In [3]:
df = df.iloc[:,1:]

In [4]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%Y%m%d')
df['membership_expire_date'] = pd.to_datetime(df['membership_expire_date'], format='%Y%m%d')
df['registration_init_time'] = pd.to_datetime(df['registration_init_time'], format='%Y%m%d')

In [5]:
df['payment_method_id'] = pd.Categorical(df['payment_method_id'])
df['is_auto_renew'] = pd.Categorical(df['is_auto_renew'])
df['city'] = pd.Categorical(df['city'])
df['gender'] = pd.Categorical(df['gender'])
df['registered_via'] = pd.Categorical(df['registered_via'])
df['is_churn'] = pd.Categorical(df['is_churn'])
df['is_cancel'] = pd.Categorical(df['is_cancel'])

In [6]:
k =(df['bd']//10)*10
b = []
for i in k:
    if i >100:
        b.append(np.nan)
    elif i < 0:
        b.append(np.nan)
    else:
        try:
            b.append(int(i))
        except:
            b.append(i)

In [7]:
df['new_bd']=b

In [8]:
df['new_bd'] = pd.Categorical(df['new_bd'])

In [9]:
df.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,city,bd,gender,registered_via,registration_init_time,is_churn,new_bd
0,IjJ2K6chuC3uAgw/wjmsInpI/b2imM01D80cQHfN2wU=,32,410,1788,1788,0,2016-01-12,2017-02-28,0,5.0,44.0,NaN,3.0,2013-10-16,0,40.0
1,/IkS3fIO/BRhlMFFErSgrtpqzedcrwnW/FUbaB1E8JM=,38,410,1788,1788,0,2016-01-04,2017-02-17,0,15.0,38.0,female,9.0,2013-10-23,1,30.0
2,bwVRgPPJnJg62tVnHJWejdeavNP8DJQdBfrhzKm6Org=,38,30,149,149,0,2017-01-09,2017-02-08,0,5.0,0.0,NaN,4.0,2015-11-13,1,0.0
3,CZ87lCEDKmbk7EXCy6Kh0zopNJxoEBTfWq9Eq9Az+28=,32,195,894,894,0,2016-07-26,2017-02-09,0,13.0,21.0,female,3.0,2012-07-11,1,20.0
4,hCqnh1Sv8tXwSFHSw4ZGql6MwI31g+ahsszfmVTAE1g=,38,410,1788,1788,0,2015-12-25,2017-02-07,0,5.0,57.0,male,3.0,2014-07-02,0,50.0


In [10]:
df['mem_duration'] = df['membership_expire_date'] - df['registration_init_time']

In [11]:
df_2 = df.iloc[:,[0,1,2,3,4,5,6,7,8,9,11,12,15,16,14]].copy()

In [12]:
plan_actual_price=[]
for i,j in zip(df_2['plan_list_price'],df_2['actual_amount_paid']):
    if i==j:
        plan_actual_price.append(0)
    if i>j:
        plan_actual_price.append(-1)
    if i<j:
        plan_actual_price.append(1)

In [13]:
df_2['plan_actual_price'] = plan_actual_price
df_2['plan_actual_price'] = pd.Categorical(df_2['plan_actual_price'])

In [14]:
df_3 =df_2.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,14]].copy()

In [15]:
df_3.shape

(992931, 16)

In [16]:
df_3['mem_duration'] = (df_3['mem_duration'].values / np.timedelta64(1, 'D')).astype(int)

In [17]:
df_3.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,city,gender,registered_via,new_bd,mem_duration,plan_actual_price,is_churn
0,IjJ2K6chuC3uAgw/wjmsInpI/b2imM01D80cQHfN2wU=,32,410,1788,1788,0,2016-01-12,2017-02-28,0,5.0,NaN,3.0,40.0,1231,0,0
1,/IkS3fIO/BRhlMFFErSgrtpqzedcrwnW/FUbaB1E8JM=,38,410,1788,1788,0,2016-01-04,2017-02-17,0,15.0,female,9.0,30.0,1213,0,1
2,bwVRgPPJnJg62tVnHJWejdeavNP8DJQdBfrhzKm6Org=,38,30,149,149,0,2017-01-09,2017-02-08,0,5.0,NaN,4.0,0.0,453,0,1
3,CZ87lCEDKmbk7EXCy6Kh0zopNJxoEBTfWq9Eq9Az+28=,32,195,894,894,0,2016-07-26,2017-02-09,0,13.0,female,3.0,20.0,1674,0,1
4,hCqnh1Sv8tXwSFHSw4ZGql6MwI31g+ahsszfmVTAE1g=,38,410,1788,1788,0,2015-12-25,2017-02-07,0,5.0,male,3.0,50.0,951,0,0


In [18]:
tmp = []
for i in df_3['gender']:
    if i == 'female':
        tmp.append(0)
    elif i == 'male':
        tmp.append(1)
    else:
        tmp.append(i)
    

In [19]:
df_3['gender']=tmp
df_3['gender'] = pd.Categorical(df_3['gender'])

In [20]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992931 entries, 0 to 992930
Data columns (total 16 columns):
msno                      992931 non-null object
payment_method_id         992931 non-null category
payment_plan_days         992931 non-null int64
plan_list_price           992931 non-null int64
actual_amount_paid        992931 non-null int64
is_auto_renew             992931 non-null category
transaction_date          992931 non-null datetime64[ns]
membership_expire_date    992931 non-null datetime64[ns]
is_cancel                 992931 non-null category
city                      877161 non-null category
gender                    391692 non-null category
registered_via            877161 non-null category
new_bd                    876806 non-null category
mem_duration              992931 non-null int64
plan_actual_price         992931 non-null category
is_churn                  992931 non-null category
dtypes: category(9), datetime64[ns](2), int64(4), object(1)
memory usage: 6

In [21]:
maxdate = max(df_3['transaction_date'])

In [22]:
tmp = maxdate - df_3['transaction_date']
trdate_diff_from_maxdate = (tmp.values / np.timedelta64(1, 'D')).astype(int)

In [23]:
df_3['trdate_diff_from_maxdate']=trdate_diff_from_maxdate

In [24]:
tmp=[]
for i in df_3['mem_duration']:
    if i == -2147483648:
        tmp.append(np.nan)
    else:
        tmp.append(i)
        
df_3['mem_duration'] = tmp

In [25]:
df_4 = df_3.drop(columns=['transaction_date', 'membership_expire_date'])

In [26]:
df_5 = df_4.dropna()

In [27]:
df_5.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,14]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391421 entries, 1 to 992928
Data columns (total 13 columns):
payment_method_id           391421 non-null category
payment_plan_days           391421 non-null int64
plan_list_price             391421 non-null int64
actual_amount_paid          391421 non-null int64
is_auto_renew               391421 non-null category
is_cancel                   391421 non-null category
city                        391421 non-null category
gender                      391421 non-null category
registered_via              391421 non-null category
new_bd                      391421 non-null category
mem_duration                391421 non-null int64
plan_actual_price           391421 non-null category
trdate_diff_from_maxdate    391421 non-null int64
dtypes: category(8), int64(5)
memory usage: 20.9 MB


In [28]:
df_5.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,city,gender,registered_via,new_bd,mem_duration,plan_actual_price,is_churn,trdate_diff_from_maxdate
1,/IkS3fIO/BRhlMFFErSgrtpqzedcrwnW/FUbaB1E8JM=,38,410,1788,1788,0,0,15.0,0.0,9.0,30.0,1213,0,1,421
3,CZ87lCEDKmbk7EXCy6Kh0zopNJxoEBTfWq9Eq9Az+28=,32,195,894,894,0,0,13.0,0.0,3.0,20.0,1674,0,1,217
4,hCqnh1Sv8tXwSFHSw4ZGql6MwI31g+ahsszfmVTAE1g=,38,410,1788,1788,0,0,5.0,1.0,3.0,50.0,951,0,0,431
5,ezv3cTcVQSMKv2IyVSQrIwfqjm4A3BVcMVvso6cVpwk=,32,180,536,536,0,0,21.0,0.0,9.0,20.0,183,0,0,207
6,RqoXgu0eHNS8VT2nFmhYTFIqanKgoWDILzZcWDEPusY=,32,180,536,536,0,0,21.0,0.0,3.0,20.0,1031,0,0,210


In [29]:
df_5.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,city,gender,registered_via,new_bd,mem_duration,plan_actual_price,is_churn,trdate_diff_from_maxdate
1,/IkS3fIO/BRhlMFFErSgrtpqzedcrwnW/FUbaB1E8JM=,38,410,1788,1788,0,0,15.0,0.0,9.0,30.0,1213,0,1,421
3,CZ87lCEDKmbk7EXCy6Kh0zopNJxoEBTfWq9Eq9Az+28=,32,195,894,894,0,0,13.0,0.0,3.0,20.0,1674,0,1,217
4,hCqnh1Sv8tXwSFHSw4ZGql6MwI31g+ahsszfmVTAE1g=,38,410,1788,1788,0,0,5.0,1.0,3.0,50.0,951,0,0,431
5,ezv3cTcVQSMKv2IyVSQrIwfqjm4A3BVcMVvso6cVpwk=,32,180,536,536,0,0,21.0,0.0,9.0,20.0,183,0,0,207
6,RqoXgu0eHNS8VT2nFmhYTFIqanKgoWDILzZcWDEPusY=,32,180,536,536,0,0,21.0,0.0,3.0,20.0,1031,0,0,210


In [30]:
scaler = StandardScaler()

In [31]:
tmp = pd.DataFrame(scaler.fit_transform(df_5.iloc[:,[2,3,4,11,14]]))
tmp2 = df_5.iloc[:,[1,5,6,7,8,9,10,12]].copy()

tmp.columns = df_5.iloc[:,[2,3,4,11,14]].columns
df6 = pd.concat([tmp,tmp2.reset_index(drop=True)],axis = 1)

/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [32]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391421 entries, 0 to 391420
Data columns (total 13 columns):
payment_plan_days           391421 non-null float64
plan_list_price             391421 non-null float64
actual_amount_paid          391421 non-null float64
mem_duration                391421 non-null float64
trdate_diff_from_maxdate    391421 non-null float64
payment_method_id           391421 non-null category
is_auto_renew               391421 non-null category
is_cancel                   391421 non-null category
city                        391421 non-null category
gender                      391421 non-null category
registered_via              391421 non-null category
new_bd                      391421 non-null category
plan_actual_price           391421 non-null category
dtypes: category(8), float64(5)
memory usage: 17.9 MB


In [33]:
# train test split
train_vars, test_vars, train_labels, test_labels = train_test_split(df6, df_5['is_churn'], test_size=0.3, random_state=0)

In [34]:
%%time
RFT = RandomForestClassifier(n_estimators =100)
RFT.fit(train_vars, train_labels) 
pred_labels = RFT.predict(test_vars)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 10805 out of 117427
Accuracy: 0.91
CPU times: user 26.4 s, sys: 108 ms, total: 26.5 s
Wall time: 26.5 s


In [35]:
from sklearn.model_selection import cross_val_predict
cv_pred = cross_val_predict(RFT,df6, df_5['is_churn'], cv=10)

In [36]:
print('Misclassified samples: {} out of {}'.format((cv_pred != df_5['is_churn']).sum(),len(df_5)))
print('CV Accuracy: %.2f' % accuracy_score(df_5['is_churn'], cv_pred))

Misclassified samples: 36855 out of 391421
CV Accuracy: 0.91


In [37]:
train_vars.columns

Index(['payment_plan_days', 'plan_list_price', 'actual_amount_paid',
       'mem_duration', 'trdate_diff_from_maxdate', 'payment_method_id',
       'is_auto_renew', 'is_cancel', 'city', 'gender', 'registered_via',
       'new_bd', 'plan_actual_price'],
      dtype='object')

In [38]:
## rf 변수 중요도
rf_imp = pd.DataFrame([train_vars.columns,RFT.feature_importances_]).transpose()
rf_imp.sort_values(by = 1,ascending = False)

,0,1
3,mem_duration,0.341833
4,trdate_diff_from_maxdate,0.296168
8,city,0.10507
6,is_auto_renew,0.0713556
11,new_bd,0.0443075
5,payment_method_id,0.0313793
0,payment_plan_days,0.0247416
2,actual_amount_paid,0.0222264
1,plan_list_price,0.0212363
10,registered_via,0.0190502


In [39]:
##SCM 오래걸림 ,,,,
'''%%time
sinvecma = SVC()
sinvecma.fit(train_vars, train_labels) 
pred_labels = sinvecma.predict(test_vars)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))'''

"%%time\nsinvecma = SVC()\nsinvecma.fit(train_vars, train_labels) \npred_labels = sinvecma.predict(test_vars)\nprint('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))\nprint('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))"

In [40]:
%%time
knn=KMeans(n_clusters=2)
knn.fit(train_vars, train_labels) 
pred_labels = knn.predict(test_vars)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 64100 out of 117427
Accuracy: 0.45
CPU times: user 2.34 s, sys: 2.6 s, total: 4.94 s
Wall time: 1.04 s


In [41]:
cv_pred_knn = cross_val_predict(knn,df6, df_5['is_churn'], cv=10)

In [42]:
print('Misclassified samples: {} out of {}'.format((cv_pred_knn != df_5['is_churn']).sum(),len(df_5)))
print('CV Accuracy: %.2f' % accuracy_score(df_5['is_churn'], cv_pred_knn))

Misclassified samples: 198165 out of 391421
CV Accuracy: 0.49


In [43]:
%%time
sgd=SGDClassifier()
sgd.fit(train_vars, train_labels) 
sgd_pred_labels = sgd.predict(test_vars)
print('Misclassified samples: {} out of {}'.format((sgd_pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, sgd_pred_labels))

/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Misclassified samples: 10418 out of 117427
Accuracy: 0.91
CPU times: user 426 ms, sys: 228 ms, total: 654 ms
Wall time: 359 ms


In [44]:
cv_pred_sgd = cross_val_predict(sgd,df6, df_5['is_churn'], cv=10)

/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they

In [45]:
print('Misclassified samples: {} out of {}'.format((cv_pred_knn != df_5['is_churn']).sum(),len(df_5)))
print('CV Accuracy: %.2f' % accuracy_score(df_5['is_churn'], cv_pred_sgd))

Misclassified samples: 198165 out of 391421
CV Accuracy: 0.83


In [46]:
%%time
lr = LogisticRegression(C=0.1, penalty='l1', random_state=0) # Lasso regression
lr.fit(train_vars, train_labels) # 학습
lr_pred_labels = lr.predict(test_vars)
print('Misclassified samples: {} out of {}'.format((lr_pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, lr_pred_labels))

/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Misclassified samples: 10307 out of 117427
Accuracy: 0.91
CPU times: user 5.37 s, sys: 258 ms, total: 5.63 s
Wall time: 5.3 s


In [47]:
cv_pred_lr = cross_val_predict(lr,df6, df_5['is_churn'], cv=10)

/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jydev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 

In [48]:
print('Misclassified samples: {} out of {}'.format((cv_pred_knn != df_5['is_churn']).sum(),len(df_5)))
print('CV Accuracy: %.2f' % accuracy_score(df_5['is_churn'], cv_pred_lr))

Misclassified samples: 198165 out of 391421
CV Accuracy: 0.91


In [49]:
#coef
lr_imp = pd.DataFrame([train_vars.columns,lr.coef_[0]]).transpose()
lr_imp.sort_values(by = 1,ascending = False)


,0,1
7,is_cancel,1.78354
0,payment_plan_days,0.359556
2,actual_amount_paid,0.0438605
5,payment_method_id,0.0056867
9,gender,0.00410683
11,new_bd,0.00188938
8,city,-0.00414054
10,registered_via,-0.0125387
4,trdate_diff_from_maxdate,-0.0397884
3,mem_duration,-0.0751204


In [50]:
#coef 절대값 변수 정규화 안되어있어서 무의미
lr_imp = pd.DataFrame([train_vars.columns,abs(lr.coef_[0])]).transpose()
lr_imp.sort_values(by = 1,ascending = False)

,0,1
6,is_auto_renew,2.19442
7,is_cancel,1.78354
12,plan_actual_price,0.417923
0,payment_plan_days,0.359556
1,plan_list_price,0.21184
3,mem_duration,0.0751204
2,actual_amount_paid,0.0438605
4,trdate_diff_from_maxdate,0.0397884
10,registered_via,0.0125387
5,payment_method_id,0.0056867


In [51]:
##ss ans sp

In [52]:
RFT.classes_

array([0, 1])

In [65]:
def cm(predicted_values):
    id_list = pd.DataFrame(df6.index)  
    preds =  pd.DataFrame(predicted_values)
    actual = pd.DataFrame(df_5['is_churn'])
    pred_df = pd.concat([id_list,actual,preds],axis=1)
    pred_df.columns = ['id','Actual Class','Predicted Class']
    cm = pd.crosstab(index=pred_df['Actual Class'], columns=pred_df['Predicted Class'])
    return cm

CLASS_LIST = RFT.classes_

def ss_and_sp(cm):
    first_index = CLASS_LIST[1]
    second_index =CLASS_LIST[0]  
    sensitivity = cm[first_index][first_index] / (cm[first_index][first_index]+ cm[second_index][first_index]) 
    specificity = cm[second_index][second_index] / (cm[first_index][second_index]+ cm[second_index][second_index]) 
    return sensitivity, specificity

def pr_and_fnr(cm):
    first_index = CLASS_LIST[1]
    second_index =CLASS_LIST[0]  
    precision = cm[first_index][first_index] / (cm[first_index][first_index]+ cm[first_index][second_index]) 
    fnr = cm[second_index][first_index] / (cm[second_index][first_index]+ cm[first_index][first_index]) 
    return precision, fnr

In [61]:
    first_index = CLASS_LIST[1]
    second_index =CLASS_LIST[0]  

In [67]:
cm(cv_pred_lr)

Predicted Class,0.0,1.0
Actual Class,,
0,119892,1466
1,18781,880


In [55]:
print("CV ss,sp of RandomForestClassifier: {:.3f}, {:.3f}".format(ss_and_sp(cm(cv_pred))[0], ss_and_sp(cm(cv_pred))[1]))
print("CV ss,sp of KNN: {:.3f}, {:.3f}".format(ss_and_sp(cm(cv_pred_knn))[0], ss_and_sp(cm(cv_pred_knn))[1]))
print("CV ss,sp of SGD: {:.3f}, {:.3f}".format(ss_and_sp(cm(cv_pred_sgd))[0], ss_and_sp(cm(cv_pred_sgd))[1]))
print("CV ss,sp of Linear Regression: {:.3f}, {:.3f}".format(ss_and_sp(cm(cv_pred_lr))[0], ss_and_sp(cm(cv_pred_lr))[1]))

CV ss,sp of RandomForestClassifier: 0.095, 0.965
CV ss,sp of KNN: 0.479, 0.487
CV ss,sp of SGD: 0.113, 0.872
CV ss,sp of Linear Regression: 0.045, 0.988


In [76]:
(df_5['is_churn'].astype(int)).sum()/len(df_5['is_churn'])

0.08744037749635303

In [66]:
print("CV pr,fnr of RandomForestClassifier: {:.3f}, {:.3f}".format(pr_and_fnr(cm(cv_pred))[0], pr_and_fnr(cm(cv_pred))[1]))
print("CV pr,fnr of KNN: {:.3f}, {:.3f}".format(pr_and_fnr(cm(cv_pred_knn))[0], pr_and_fnr(cm(cv_pred_knn))[1]))
print("CV pr,fnr of SGD: {:.3f}, {:.3f}".format(pr_and_fnr(cm(cv_pred_sgd))[0], pr_and_fnr(cm(cv_pred_sgd))[1]))
print("CV pr,fnr of Linear Regression: {:.3f}, {:.3f}".format(pr_and_fnr(cm(cv_pred_lr))[0], pr_and_fnr(cm(cv_pred_lr))[1]))

CV pr,fnr of RandomForestClassifier: 0.303, 0.905
CV pr,fnr of KNN: 0.131, 0.521
CV pr,fnr of SGD: 0.125, 0.887
CV pr,fnr of Linear Regression: 0.375, 0.955


In [56]:
print("---RandomForestClassifier---")
print('Misclassified samples: {} out of {}'.format((cv_pred != df_5['is_churn']).sum(),len(df_5)))
print('CV Accuracy: %.2f\n' % accuracy_score(df_5['is_churn'], cv_pred))

print("---KNN---")
print('Misclassified samples: {} out of {}'.format((cv_pred_knn != df_5['is_churn']).sum(),len(df_5)))
print('CV Accuracy: %.2f\n' % accuracy_score(df_5['is_churn'], cv_pred_knn))

print("---SGD---")
print('Misclassified samples: {} out of {}'.format((cv_pred_sgd != df_5['is_churn']).sum(),len(df_5)))
print('CV Accuracy: %.2f\n' % accuracy_score(df_5['is_churn'], cv_pred_sgd))

print("---LR---")
print('Misclassified samples: {} out of {}'.format((cv_pred_lr != df_5['is_churn']).sum(),len(df_5)))
print('CV Accuracy: %.2f\n' % accuracy_score(df_5['is_churn'], cv_pred_lr))

---RandomForestClassifier---
Misclassified samples: 36855 out of 391421
CV Accuracy: 0.91

---KNN---
Misclassified samples: 198165 out of 391421
CV Accuracy: 0.49

---SGD---
Misclassified samples: 68387 out of 391421
CV Accuracy: 0.83

---LR---
Misclassified samples: 34494 out of 391421
CV Accuracy: 0.91



In [57]:
from sklearn.metrics import f1_score
print('micro f1 score of RFC: %.2f\n' %f1_score(df_5['is_churn'], cv_pred, average='micro'))
print('micro f1 score of KNN: %.2f\n' %f1_score(df_5['is_churn'], cv_pred_knn, average='micro'))
print('micro f1 score of SGD: %.2f\n' %f1_score(df_5['is_churn'], cv_pred_sgd, average='micro'))
print('micro f1 score of LR: %.2f\n' %f1_score(df_5['is_churn'], cv_pred_lr, average='micro'))

micro f1 score of RFC: 0.91

micro f1 score of KNN: 0.49

micro f1 score of SGD: 0.83

micro f1 score of LR: 0.91



In [58]:
cm(cv_pred)[0][0]

117068

In [59]:
1869/ (1869+17792 )

0.09506128884593866

In [60]:
117045 / (117045+4313)

0.9644605217620593